In [1]:
import os
import time
import requests # used for making HTTP requests
from getpass import getpass # password input function

In [2]:
def pathExists(path):
    if os.path.exists(path):
        return True
    else:
        print(f"Invalid Path: {path}")
        return False

In [3]:
# Let's first create a working directory for this analysis and change into it
path = '/home/jovyan/notebooks/ASF/GEOS_657_Labs'
if pathExists(path):
    os.chdir(path)
    !mkdir -p learn_Vertex_API_data
    path = f"{path}/learn_Vertex_API_data"
    if pathExists(path):
        os.chdir(path)
print(f"current directory: {os.getcwd()}")

current directory: /home/jovyan/notebooks/ASF/GEOS_657_Labs/learn_Vertex_API_data


In [4]:
def download_file(url, username=None, password=None):
    #https://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py
    local_filename = os.path.basename(url)
    r = requests.get(url, stream=True)   # NOTE the stream=True parameter
    if r.status_code == 401:
        pwd = getpass('Password for {}: '.format(username))
        r = requests.get(r.url, stream=True, auth=(username,pwd))
    totalLength = r.headers.get('content-length')
    print(f"Downloading {url}")
    start = time.perf_counter()
    with open(local_filename, 'wb') as f:
        if r is None:
            f.write(r.content)
        else:
            dl = 0
            totalLength = int(totalLength)
            for chunk in r.iter_content(chunk_size=1024):
                dl += len(chunk)
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    done = int(50 * dl / totalLength)
                    print("\r[%s%s] %s bps, %s%%    " % ('=' * done, ' ' * (50-done), dl//(time.perf_counter() - start), int((100*dl)/totalLength)), end='\r', flush=True)    
        if os.stat(local_filename).st_size < 5000:
            print('\nDownload failed!\nPlease check your username, password, and download URL.\n')
            return
        else:
            print('\nDone\n')
            return local_filename

In [13]:
NEDusr = 'aflewandowski'
NEDpass = getpass()

········


## Get a granule URL using the granule name:

In [44]:
def get_vertex_granule_URL(granule_name=None):
    vertex_API_URL = "https://api.daac.asf.alaska.edu/services/search/param"
    r = requests.get(vertex_API_URL, 
                     params=[('granule_list', 'E2_15295_STD_F469'), 
                             ('output', 'csv')])
    if r.status_code == 401:
        pwd = getpass('Password for {}: '.format(username))
        r = requests.get(r.url, stream=True, auth=(username,pwd))
    return r

In [46]:
response = get_vertex_granule_URL('E2_15295_STD_F469')
print(response.content)

b'"Granule Name","Platform","Sensor","Beam Mode","Beam Mode Description","Orbit","Path Number","Frame Number","Acquisition Date","Processing Date","Processing Level","Start Time","End Time","Center Lat","Center Lon","Near Start Lat","Near Start Lon","Far Start Lat","Far Start Lon","Near End Lat","Near End Lon","Far End Lat","Far End Lon","Faraday Rotation","Ascending or Descending?","URL","Size (MB)","Off Nadir Angle","Stack Size","Baseline Perp.","Doppler","GroupID"\n"E2_15295_STD_F469","ERS-2","SAR","STD","ERS-1,ERS-2,JERS-1,SEASAT,SMAP Standard Beam SAR","15295","411","3742","1998-03-24T14:47:43Z","2010-12-06T12:55:10Z","L1","1998-03-24T14:47:27Z","1998-03-24T14:47:43Z","-7.6074","-68.2281","-7.2211","-67.6813","-7.0217","-68.5575","-8.1929","-67.897","-7.9922","-68.7767","NA","DESCENDING","https://datapool.asf.alaska.edu/L1/E2/E2_15295_STD_F469.zip","50.45","-1","1","0","0","NA"\n"E2_15295_STD_F469","ERS-2","SAR","STD","ERS-1,ERS-2,JERS-1,SEASAT,SMAP Standard Beam SAR","15295","411

In [ ]:
# Now, let's actually download the file:
#url = 'https://datapool.asf.alaska.edu/L1.0/A3/ALPSRP185270680-L1.0.zip'
filename = download_file(url, username=NEDusr, password=NEDpass)